# Dati
ci occuperemo di dati tabellari:
* ogni riga è un record (info che riguarda una certa istanza)
* per ogni riga ho un certo numero di colonne che chiameremo features -> caratteristiche che definiscono un'istanza.

tipicamente ogliamo fare un task di predizione.
Per esempio: ogni colonna è un corso, e sulle righe i voti per studente. il task è predire se il tizio si laurea con la lode [chiamato variabile dipedente, label, etichetta e altri nomi]. 
Predico basandomi sulla base di uno storico dei dati.
Possono esistere tantissimi modelli predittivi.
Tipicamente si costruisce il modello guardando lo storico (stundenti passati), si cerca un pattern. Sto storico si chiama **training set**. Queste sono informazioni che si usano per allenare il modello. Una volta che ho il modello, posso applicarlo a nuove istanze.
L'obbiettico è stduio il passato per poi fare predizioni su studenti mai visti prima.

L'etichetta in questo caso è di "classficazione": si chiama così l'output binario.

Per capire se il modello è buono o meno, uso un altro insieme di dati con le stesse features di prima, ma con istanze mai viste. Le diamo in pasto al modello e controlliamo se il risultato predetto è uguale a quello reale.

La rappresentatività delle features è fondamentale per la riuscita del predittore (per esempio se baso lode/non lode sulle features sesso e anno di nascita, avrò predittori sbagliati perché le features non sono adatte a rappresentarmi il problema).

Se l'etichetta è continua (per esmpio il voto invece della lode), cambia anche la valutazione del modello, perché sbaglia si/no è meno utile di sbaglia tanto/poco. 
Misuro lo scarto (dipende dalla feature "quale" scarto, se valore assoluto, sqm, scarto logaritmico etc). 
Lo scarto deve aiutarmi a distinguere un modello dall'altro e decidere quale è migliore.

Il "learning"/"training" viene detto **supervised** perché l'etichetta è data, e si differenzia quindi dall' "unsupervised learning" AKA clustering

# Supervised Learning
è uno dei più semplici algoritmi di machine learning.
input: dati tabellari
## k-Nearest-Neighbor Classifier
in pratica, confronta la tupla con quelle più simile nel dataset per fare le predizioni. si dice che impara per analogia.
### Train e Test
come visto prima, per verificare la validità di un modello, è necessario testarlo. 
Prendiamo quindi il dataset e lo splittiamo. Una parte delle righe disponibili viene usato per il training, il rimanente viene lasciato lì per l'esecuzione dei test.
In genere si divide in 2/3 training e 1/3 test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(y, test_size=33, random_state=42)

### accuratezza
dicevamo che il caloclo dell'accuratezza è diverso tra i casi di label continue o discrete. In sto caso usiamo la *distanza* della predizione dal valore corretto.
in `sklearn` abbiamola classe `KNeighborsClassifier`. È un modello classificatore cge sceglie un gruppo a seconda della frequenza di vicini che ha

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Create the classifier
kNN = KNeighborsClassifier(n_neighbors=10)

# Train the classifier
kNN.fit(X_train,y_train)

# Generate predictions
y_pred = kNN.predict(X_test)

# compute Accuracy
acc = accuracy_score(y_true=y_test, y_pred=y_pred)
print (f"Accuracy {acc:.3f}")

### Normalizzazione
basandosi sulle distanze tra gli oggetti si è soggetti ad errori in quanto non si tiene conto del peso di ogni feature.
Per esempio, i numeri possono essere più o meno alti a seconda dell'unità di misura. se misuro la distanza di età di due persone in secondi, mi vengono numeri enormi, e magari le distanze di tutte le altre feature diventano trascurabili a confronto. Le feature hanno "peso" confrontabile o sensato?
Per risolvere questo si normalizza. tra 0 e 1 il range di ogni feature: `MinMaxScaler()`.
Non solo, se so che tutti i dati hanno distribuzione gaussiana, posso normalizzarli con `StandardScaler()`.

# Wine dataset (dal file 03 DWM)

Url: http://archive.ics.uci.edu/ml/datasets/Wine

These data are the results of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines.


The attributes are:

1. Quality (1-3)
2. Alcohol
3. Malic acid
4. Ash
5. Alcalinity of ash
6. Magnesium
7. Total phenols
8. Flavanoids
9. Nonflavanoid phenols
10. Proanthocyanins
11. Color intensity
12. Hue
13. OD280/OD315 of diluted wines
14. Proline


In [ ]:
import numpy as np
import pandas as pd

data_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
df = pd.read_csv(data_url, header=None)

da qui posso usare le funzioni viste lezione scorsa

In [ ]:
df.head()
df.describe().T
df.info()

# data preparation
# convert to float to have precise and homogenous computation
dataset = df.astype(float)
print("dataset shape", dataset.shape)

# get features by removing class label
# remove id
X = dataset.loc[:,1:]
print("X shape", X.shape)

# get class label
y = dataset.loc[:,0]
print("y shape", y.shape)

### split train/test


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

print (X_train.shape, X_test.shape)

### Normalizzazione

In [ ]:
from sklearn.preprocessing import MinMaxScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

scaler = MinMaxScaler()
scaler.fit(X_train) # lo scaling si fa sul training set

for k in range(1,11): # per trovare il miglior valore di k 

    kNN = KNeighborsClassifier(n_neighbors=k)
    kNN.fit( scaler.transform(X_train), y_train ) # nel fare il
        # modello, si appliica lo scaling trovato .transform sul training dataset
    y_pred = kNN.predict( scaler.transform(X_test) ) # uso lo scaler trovato
        # su X_train per fare la trasformazione del dataset di test

    # compute Accuracy
    acc = accuracy_score(y_true=y_test, y_pred=y_pred)
    print ("k: {:2d} | Accuracy {:.3f}".format(k,acc) )